# An Intro to Polylines - The Interpolated Spline

I do a lot of work in the CAD world and one of my tools is the spline, it
essentially gives you a way to tell the computer, *"here's a bunch of points,
please draw a smooth line that passes through them"*.

The term, *"spline"*, originally came from ship builders who would use flexible
strips of wood held using pegs to draw smooth curves, but nowadays the term can
refer to a variety of curves.

In an effort to explore the mathematics behind the various types of spline, I
thought I'd try to implement them myself and add them to [`arcs`][arcs], a
Rust CAD engine I've been playing around with in my spare time.

Here are some of the splines we can choose from:

- [Hermite Spline](https://en.wikipedia.org/wiki/Cubic_Hermite_spline) - the
  mathematical function you get when trying to minimise the "tension" in a
  flexible curve passing
- [B-Spline](https://en.wikipedia.org/wiki/B-spline)
- [NURBS](https://en.wikipedia.org/wiki/Non-uniform_rational_B-spline) - The
  general form of a *B-Spline*, used all over the place in CAM
- [Bézier Curve](https://en.wikipedia.org/wiki/B%C3%A9zier_curve) - sibling of
  the *B-Spline*, often used in fonts
- [Interpolated Cubic/Quadratic Spline][wiki] - what you get after fitting a
  polynomial to the curve

As the easiest for me to wrap my head around, I thought I'd start with the
[*Interpolated Spline*][wiki]. 

I'll try not to go too deeply into the math. However, considering computational 
geometry (the study of drawing things and doing geometry inside a computer) 
is literally a field of mathematics, it's not possible to avoid it altogether.

[arcs]: https://github.com/Michael-F-Bryan/arcs
[wiki]: https://en.wikipedia.org/wiki/Polynomial_interpolation

In [2]:
:dep plotters = { git = "https://github.com/38/plotters", features = ["evcxr"] }
:dep derive_more = "0.99.7"
:dep nalgebra = "0.21.0"

use plotters::prelude::*;
use nalgebra::base::{Matrix, DMatrix, dimension::*};

In [3]:
use std::fmt::{self, Display, Formatter};
use derive_more::*;

#[derive(Copy, Clone, Debug, PartialEq, Add, Mul, Sum, AddAssign, MulAssign)]
pub struct Point {
    x: f64,
    y: f64,
}

impl Point {
    pub const fn new(x: f64, y: f64) -> Self {
        Point { x, y }
    }
    
    pub const fn zero() -> Self {
        Point::new(0.0, 0.0)
    }
    
    pub const fn tuple(self) -> (f64, f64) {
        (self.x, self.y)
    }
}

impl Display for Point {
    fn fmt(&self, f: &mut Formatter<'_>) -> fmt::Result {
        write!(f, "(")?;
        Display::fmt(&self.x, f)?;
        write!(f, ", ")?;
        Display::fmt(&self.y, f)?;
        write!(f, ")")?;

        Ok(())
    }
}

In [4]:
use plotters::coord::RangedCoordf64;

fn draw_points< 'a, P, D, F>(
    ctx: &'a mut ChartContext<'a, D, RangedCoord<RangedCoordf64, RangedCoordf64>>, 
    points: P, 
    mut getter: F) -> Result<(), Box<dyn std::error::Error + 'a>> 
where D: DrawingBackend + 'a,
      P: IntoIterator<Item = Point>,
      F: FnMut(usize, Point) -> (f64, f64),
      D::ErrorType: 'a,
{
    let mut points: Vec<_> = points.into_iter().enumerate().map(|(i, p)| getter(i, p)).collect();
    ctx.draw_series(points.iter().map(|p| Circle::new(*p, 3, RED.filled())))?;

    let path = PathElement::new(points.iter().copied().collect::<Vec<_>>(), RED.filled());
    ctx.plotting_area().draw(&path)?;
    
    Ok(())
}

## The General Idea


Say you have a bunch of points and you'd like a smooth curve that passes 
through them. 

In [5]:
let points = vec![
    Point::new(0.0, 0.0),
    Point::new(0.7, 1.2),
    Point::new(1.1, 3.1),
    Point::new(4.0, 2.1),
    Point::new(2.5, -1.0),
];

By far the easiest way to draw this is to just connect the dots, like so.

In [6]:
evcxr_figure((640, 480), |root| {
   let mut ctx = ChartBuilder::on(&root)
    .x_label_area_size(40)
    .y_label_area_size(40)
    .build_ranged(-1.0f64.. 5.0f64, -2.0f64..4.0f64)?;
    
    ctx.configure_mesh().draw()?;
    
    fn type_name<T>(_: &T) -> &'static str {
        std::any::type_name::<T>()
    }
    
    ctx.draw_series(points.iter().map(|p| Circle::new(p.tuple(), 3, RED.filled())))?;

    let path = PathElement::new(points.iter().map(|p| p.tuple()).collect::<Vec<_>>(), RED.filled());
    ctx.plotting_area().draw(&path)?;
        
    Ok(())
})

-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0

While it may feel a bit naive, for a lot of applications this can be quite adequate.
If the points are close together or don't change very much, the difference 
between the "actual" curve which smoothly passes through each point and our linear 
approximation will be pretty small.

But what if we *do* want a smooth curve, and a bunch of connected lines just won't
cut it?

Data scientiests can often get away with using linear approximation over something
more complex because the numerical error tends to be pretty low, but if you were an
engineer drawing the shape of a seat or cushion its a lot harder to get away with
the sharp changes in direction linear approximation will give you.

There is a mathematical technique referred to as [Interpolation][interp] which 
looks at some data points and uses trends to estimate the shape of the curve passing
through them.

Ideally this interpolation process would give us a mathematical function which

1. Is continuous - there are no discontinuties (i.e. parts where the curve "jumps" 
   between two locations instantaneously)
2. Is smooth - the slope of the line should also be continuous
3. Is easy to compute and work with - this makes the math nicer and leads to better
   performance when calculated on a computer

There are many families of mathematical function, but [polynomials][poly] best fit
our criteria.

However, you'll notice something odd when trying to fit a polynomial to our previous
curve... There's no possible way we can create such a polynomial because it fails 
[the vertical line test][vertical-test].

Luckily mathematics already has a tool for dealing with this. Instead of creating a
single polynomial which, given some `x` position will provide the `y` location on the 
curve, we can introduce an intermediate variable, `t` (I normally think of it as the
percentage along the curve or the segment number), and ask for two equations mapping 
`t → x` and `t → y`, respectively.

For example, this is what we get when applying linear approximation to the `x` and `y`
components in terms of `t`.

[interp]: https://en.wikipedia.org/wiki/Interpolation
[poly]: https://en.wikipedia.org/wiki/Polynomial
[vertical-test]: https://en.wikipedia.org/wiki/Vertical_line_test

In [7]:
evcxr_figure((640, 480), |root| {
    let areas = root.split_evenly((2,1));
    let mut charts = Vec::new();

    for (area, name) in areas.iter().zip(["X", "Y"].iter()) {
        let mut chart = ChartBuilder::on(&area)
            .caption(format!("{} Component", name), ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_ranged(-1.0f64.. 5.0f64, -2.0f64..5.0f64)?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .x_desc("t")
            .y_desc(*name)
            .axis_desc_style(("Arial", 20).into_font())
            .draw()?;
        charts.push(chart);
    }
    
   let x_ctx = &mut charts[0];    
    
    x_ctx.configure_mesh().draw()?;
    let x_points: Vec<_> = points.iter().enumerate().map(|(i, p)| (i as f64, p.x)).collect();
    x_ctx.draw_series(x_points.iter().map(|p| Circle::new(*p, 3, RED.filled())))?;
    x_ctx.plotting_area().draw(&PathElement::new(x_points, RED.filled()))?;
           
       let y_ctx = &mut charts[1];    

    y_ctx.configure_mesh().draw()?;
    let x_points: Vec<_> = points.iter().enumerate().map(|(i, p)| (i as f64, p.y)).collect();
    y_ctx.draw_series(x_points.iter().map(|p| Circle::new(*p, 3, RED.filled())))?;
    y_ctx.plotting_area().draw(&PathElement::new(x_points, RED.filled()))?;
    Ok(())
})

thread '<unnamed>' panicked at 'Drawing failure: BackendError(FontError(FontLoadError(IllFormed)))', /home/michael/.cargo/git/checkouts/plotters-6aced55a29a4ec18/dee7727/src/evcxr.rs:38:5
stack backtrace:
   0: backtrace::backtrace::libunwind::trace
             at /cargo/registry/src/github.com-1ecc6299db9ec823/backtrace-0.3.46/src/backtrace/libunwind.rs:86
   1: backtrace::backtrace::trace_unsynchronized
             at /cargo/registry/src/github.com-1ecc6299db9ec823/backtrace-0.3.46/src/backtrace/mod.rs:66
   2: std::sys_common::backtrace::_print_fmt
             at src/libstd/sys_common/backtrace.rs:78
   3: <std::sys_common::backtrace::_print::DisplayBacktrace as core::fmt::Display>::fmt
             at src/libstd/sys_common/backtrace.rs:59
   4: core::fmt::write
             at src/libcore/fmt/mod.rs:1069
   5: std::io::Write::write_fmt
             at src/libstd/io/mod.rs:1537
   6: std::sys_common::backtrace::_print
             at src/libstd/sys_common/backtrace.rs:62
   7: st

Error: Child process terminated with status: signal: 11

You can see that, by construction, it's not possible for these curves to double back
on themselves (i.e. fail the vertical line test), so we can *always* find a polynomial
that passes through each points.

# Finding our Interpolation Function

Okay, so now we know we're going to split the curve into its `x` and `y` components and 
interpolate them independently.

Given the set of points

\begin{align}
  (t_0, x_0), (t_1, x_1), (t_2, x_2), \ldots , (t_n, x_n)
\end{align}

One very simple way to do a curve fit for this is to write down a `n`-degree polynomial.

\begin{align}
  x(t) = a_0 + a_1 t + a_2 t^2 \ldots + a_n t^n
\end{align}

If you look at this equation we've got $n + 1$ unknowns (the coefficients $a_0, a_1, \ldots, a_n$)
and we also have $n + 1$ constraints (our points). That means we just need to do 
simultaneous equations to find the polynomial coefficients. It's made even easier
by the fact that each of the equations is linear so solving becomes a simple matrix
operation, and computers eat matrix operations for breakfast.

If we're just wanting to calculate this curve for the original data's x-component, 
substituting each $(t, x)$ pair into the above gives us the following system of equations:

\begin{align}
  0 &= a_0 + a_1 * 0 + a_2 * 0^2 + a_3 * 0^3 + a_4 * 0^4 \\
  0.7 &= a_0 + a_1 * 1 + a_2 * 1^2 + a_3 * 1^3 + a_4 * 1^4 \\
  1.1 &= a_0 + a_1 * 2 + a_2 * 2^2 + a_3 * 2^3 + a_4 * 2^4 \\
  4 &= a_0 + a_1 * 3 + a_2 * 3^2 + a_3 * 3^3 + a_4 * 3^4 \\
  2.5 &= a_0 + a_1 * 4 + a_2 * 4^2 + a_3 * 4^3 + a_4 * 4^4
\end{align}

Which can be converted to matrix form.

\begin{align}
  A x = b \\
  \begin{bmatrix}
    1 & 0 & 0^1 & 0^2 & 0^3 & 0^4 \\
    1 & 1 & 1^1 & 1^2 & 1^3 & 1^4 \\
    1 & 2 & 2^1 & 2^2 & 2^3 & 2^4 \\
    1 & 3 & 3^1 & 3^2 & 3^3 & 3^4 \\
    1 & 4 & 4^1 & 4^2 & 4^3 & 4^4
  \end{bmatrix}   
  \begin{bmatrix} a_0 \\ a_1 \\ a_2 \\ a_3 \\ a_4 \end{bmatrix} &=
  \begin{bmatrix} 0 \\ 0.7 \\ 1.1 \\ 4 \\ 2.5 \end{bmatrix}
\end{align}

As a bonus, there's a nice pattern for this matrix, so generating it turns into 
a couple trivial loops.

Solving to get the coefficients can be done by pre-multiplying with the 
inverse to get $x = A^{-1} b$.

In [8]:
fn matrix(unknowns: usize) -> DMatrix<f64> {
    let d = Dynamic::new(unknowns);
    
    Matrix::from_fn_generic(d, d, |row, column| if column == 0 {
        1.0
    } else {
        (row as f64).powi(column as i32)
    })
}

let a: DMatrix<f64> = matrix(5);
let b: DMatrix<f64> = DMatrix::from_column_slice(5, 1, &[0.0, 0.7, 1.1, 4.0, 2.5]);

let solution: DMatrix<f64> = a.try_inverse().unwrap() * b;
let coefficients: Vec<f64> = solution.column(0).into_iter().copied().collect();

fn polynomial(t: f64, coefficients: &[f64]) -> f64 {
    coefficients.iter()
        .enumerate()
        .map(|(i, coefficient)| coefficient * t.powi(i as i32))
        .sum()
}

println!("Coefficients: {:?}", coefficients);

for t in 0 .. 5 {
    let got = polynomial(t as f64, &coefficients);
    println!("({}, {})", t, got);
}

Coefficients: [0.0, 4.208333333333336, -5.995833333333335, 2.8916666666666666, -0.4041666666666667]
(0, 0)
(1, 0.7000000000000003)
(2, 1.099999999999996)
(3, 3.999999999999993)
(4, 2.4999999999999716)


()

In [13]:
pub fn linspace(start: f64, stop: f64, step: f64) -> impl Iterator<Item = f64> {
    let mut items = Vec::new();
    let mut current_value = start;
    
    while current_value <= stop {
        items.push(current_value);
        current_value += step;
    }
    
    items.into_iter()
}

evcxr_figure((640, 480), |root| {
   let mut ctx = ChartBuilder::on(&root)
    .x_label_area_size(40)
    .y_label_area_size(40)
    .build_ranged(-1.0f64.. 5.0f64, -0.5f64..5.0f64)?;
    
    ctx.configure_mesh().draw()?;
    
    let points = points.iter().enumerate().map(|(i, p)| (i as f64, p.x));
    ctx.draw_series(points.map(|p| Circle::new(p, 3, RED.filled())))?;
    
    let approximation: Vec<_> = linspace(0.0, 4.0, 0.05)
        .map(|t| (t, polynomial(t, &coefficients)))
        .collect();
    let path = PathElement::new(approximation, RED.filled());
    ctx.plotting_area().draw(&path)?;
        
    Ok(())
})

0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="140,399 145,383 150,369 155,358 160,349 165,341 170,335 175,330 180,327 185,324 190,323 195,323 200,323 205,325 210,326 215,328 220,331 225,334 230,337 235,340 240,343 245,345 250,348 255,350 260,353 265,354 270,356 275,356 280,357 285,357 290,356 295,354 300,352 305,349 310,346 315,342 320,337 325,331 330,325 335,318 340,311 345,302 350,293 355,284 360,274 365,263 370,252 375,240 380,228 385,216 390,203 395,190 400,177 405,164 410,151 415,138 420,126 425,113 430,101 435,89 440,79 445,68 450,59 455,50 460,43 465,37 470,32 475,29 480,27 485,27 490,30 495,34 500,41 505,50 510,62 515,76 520,94 525,115 530,139 535,167 540,199 "/>

We can even package this all up into a `Polynomial` type.

In [10]:
#[derive(Debug, Clone, PartialEq)]
pub struct Polynomial {
    coefficients: Vec<f64>,
}

impl Polynomial {
    pub fn approximating_data(points: &[f64]) -> Self 
    {
        let dim = Dynamic::new(points.len());
        
        let a = Matrix::from_fn_generic(dim, dim, |row, column| if column == 0 {
                1.0
            } else {
                (row as f64).powi(column as i32)
            });
        let b = DMatrix::from_column_slice(points.len(), 1, points);
        
        let solution = a.try_inverse().expect("Always invertible") * b;
        let coefficients = solution.column(0).into_iter().copied().collect();
        
        Polynomial { coefficients }
    }
    
    pub fn eval(&self, t: f64) -> f64 {
        let mut value = 0.0;
        
        for (i, coefficient) in self.coefficients.iter().enumerate() {
            value +=  coefficient * t.powi(i as i32);
        }
        
        value
    }
}

But something funny happens when you try to approximate more and more points.

In [57]:
fn actual_function(t: f64) -> f64 {
    20.0 * t * t
}

let sin_wave: Vec<_> = linspace(0.0, 17.0, 1.0)
    .map(actual_function)
    .collect();

let poly = Polynomial::approximating_data(&sin_wave);

println!("{:.5?}", poly);
let min = sin_wave.iter().copied().min_by(|l, r| l.partial_cmp(r).unwrap()).unwrap();
let max = sin_wave.iter().copied().max_by(|l, r| l.partial_cmp(r).unwrap()).unwrap();
let range = max - min;

evcxr_figure((640, 480), |root| {
    let upper_bound = sin_wave.len() as f64 - 1.0;
    
    let mut ctx = ChartBuilder::on(&root)
        .margin(5)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(-1.0f64.. upper_bound + 1.0, min - range * 0.1 .. max + range *0.1)?;
    
    ctx.configure_mesh().draw()?;
    
//     let points = sin_wave.iter().enumerate().map(|(i, x)| (i as f64, *x));
//     ctx.draw_series(points.map(|p| Circle::new(p, 3, RED.filled())))?;
    
    let approximation: Vec<_> = linspace(0.0, upper_bound, 0.1)
        .map(|t| (t, poly.eval(t)))
        .collect();
    ctx.draw_series(LineSeries::new(approximation, &RED))?
        .label("Approximation");
    
    let true_curve: Vec<_> = linspace(0.0, upper_bound, 0.5)
        .map(|t| (t, actual_function(t)))
        .collect();
    ctx.draw_series(LineSeries::new(true_curve, &BLUE))?
        .label("Actual");
    
    Ok(())
})

Polynomial { coefficients: [0.00000, -0.56900, 20.00629, 0.01298, -0.00157, 0.00016, -0.00002, 0.00000, -0.00000, 0.00000, 0.00000, -0.00000, 0.00000, -0.00000, 0.00000, -0.00000, 0.00000, -0.00000] }


0.0
 
 
 
1000.0
 
 
 
2000.0
 
 
 
3000.0
 
 
 
4000.0
 
 
 
5000.0
 
 
 
6000.0
 
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
16.0
 
 
 
18.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="76,398 79,398 82,398 85,398 88,397 91,397 94,397 97,397 100,397 104,397 107,396 110,396 113,396 116,396 119,395 122,395 125,395 128,394 131,394 135,393 138,393 141,392 144,392 147,391 150,391 153,390 156,389 159,389 163,388 166,387 169,387 172,386 175,385 178,384 181,383 184,383 187,382 190,381 194,380 197,379 200,378 203,377 206,376 209,375 212,374 215,373 218,372 222,370 225,369 228,368 231,367 234,366 237,364 240,363 243,362 246,360 249,359 253,357 256,356 259,355 262,353 265,352 268,350 271,349 274,347 277,345 281,344 284,342 287,340 290,339 293,337 296,335 299,333 302,332 305,330 308,328 312,326 315,324 318,322 321,320 324,318 327,316 330,314 333,312 336,310 340,308 343,306 346,304 349,302 352,300 355,297 358,295 361,293 364,291 367,288 371,286 374,283 377,281 380,279 383,276 386,274 389,271 392,269 395,266 399,264 402,261 405,258 408,256 411,253 414,250 417,248 420,245 423,242 426,239 430,237 433,234 436,231 439,228 442,225 445,222 448,219 451,216 454,213 458,210 461,207 464,204 467,201 470,198 473,195 476,191 479,188 482,185 485,182 489,178 492,175 495,172 498,168 501,165 504,162 507,158 510,155 513,151 517,148 520,144 523,141 526,137 529,133 532,130 535,126 538,123 541,119 544,115 548,111 551,108 554,104 557,100 560,96 563,92 566,88 569,84 572,80 576,77 579,73 582,69 585,64 588,60 591,56 594,52 597,48 600,44 603,40 "/>

# Cubic Spline in Rust

# Approximating

# Taking Derivatives

# A Better Approximation

In [12]:
let points = vec![
    Point::new(0.0, 0.0),
    Point::new(0.7, 1.2),
    Point::new(1.1, 3.1),
    Point::new(4.0, 2.1),
    Point::new(2.5, -1.0),
];